In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 데이터베이스 내에 넣을 데이터프레임 가공 -> 저장

# 순서 - master_crop_variety -> map_region_weather_station -> weather_daily -> factor_external -> fact_trade

In [7]:
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, date
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import re
load_dotenv()

True

In [8]:
# 사전 DB 세팅
user = 'root'
password = os.getenv("DB_pw")
host = 'localhost'    # 또는 실제 호스트
port = 3306
db = 'jikfarm_db'
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)

# master_crop_variety(품목코드) DB 삽입

In [30]:
df = pd.read_csv('test/표준코드_품목코드_cp.csv', encoding='cp949')

# 코드조합하기
def create_crop_codes(df: pd.DataFrame) -> pd.DataFrame:
    """
    주어진 DataFrame에 'item_code'와 'full_code' 컬럼을 생성합니다.

    'item_code'는 'lclsf_cd'와 'mclsf_cd'를 조합하여 생성됩니다.
    'full_code'는 'lclsf_cd', 'mclsf_cd', 'sclsf_cd'를 조합하여 생성됩니다.

    Args:
        df (pd.DataFrame): 'lclsf_cd', 'mclsf_cd', 'sclsf_cd' 컬럼을 포함하는 DataFrame.

    Returns:
        pd.DataFrame: 'item_code'와 'full_code' 컬럼이 추가된 DataFrame.
    """
    # 각 코드 컬럼이 문자열 타입인지 확인하고, 아니면 문자열로 변환합니다.
    # 이는 코드들이 숫자형으로 읽혔을 때 발생할 수 있는 오류를 방지합니다.
    for col in ['gds_lclsf_cd', 'gds_mclsf_cd', 'gds_sclsf_cd']:
        if col in df.columns:
            df[col] = df[col].astype(str)
            # 2자리 숫자로 패딩 (예: '1' -> '01')
            df[col] = df[col].str.zfill(2)
        else:
            print(f"경고: '{col}' 컬럼이 DataFrame에 없습니다. 코드 생성에 문제가 있을 수 있습니다.")
            return df # 필수 컬럼이 없으면 함수 종료

    # 'item_code' 생성: 대분류 코드 + 품목 코드
    # 예: '11' + '01' = '1101'
    df['item_code'] = df['gds_lclsf_cd'] + df['gds_mclsf_cd']

    # 'full_code' 생성: 대분류 코드 + 품목 코드 + 품종 코드
    # 예: '11' + '01' + '01' = '110101'
    df['full_code'] = df['gds_lclsf_cd'] + df['gds_mclsf_cd'] + df['gds_sclsf_cd']

    return df

df = create_crop_codes(df)

,gds_lclsf_cd,gds_lclsf_nm,gds_mclsf_cd,gds_mclsf_nm,gds_sclsf_cd,gds_sclsf_nm,item_code,full_code
0,01,미곡류,00,미곡류(일반),00,-,0100,010000
1,01,미곡류,01,벼,00,벼,0101,010100
2,01,미곡류,01,벼,01,일반계,0101,010101
3,01,미곡류,01,벼,02,통일계,0101,010102
4,01,미곡류,01,벼,03,햇일반계,0101,010103
...,...,...,...,...,...,...,...,...
2066,14,산채류,99,기타,14,삼잎국화나물,1499,149914
2067,14,산채류,99,기타,15,어수리나물,1499,149915
2068,14,산채류,99,기타,16,삿갓나물,1499,149916
2069,14,산채류,99,기타,17,병풍취,1499,149917


In [31]:
df.drop_duplicates(inplace=True)
df.to_csv('test/표준코드_품목코드_cp2.csv', encoding='cp949')

In [12]:
df = pd.read_csv('datasets//표준코드_품목코드_utf8_sig_3.csv', encoding='utf8')
df = df.iloc[:2071]

In [13]:
df

,gds_lclsf_cd,gds_lclsf_nm,gds_mclsf_cd,gds_mclsf_nm,gds_sclsf_cd,gds_sclsf_nm,item_code,full_code
0,01,미곡류,00,미곡류(일반),-0,-,0100,0100-0
1,01,미곡류,01,벼,00,벼,0101,010100
2,01,미곡류,01,벼,01,일반계,0101,010101
3,01,미곡류,01,벼,02,통일계,0101,010102
4,01,미곡류,01,벼,03,햇일반계,0101,010103
...,...,...,...,...,...,...,...,...
2066,14,산채류,99,기타,14,삼잎국화나물,1499,149914
2067,14,산채류,99,기타,15,어수리나물,1499,149915
2068,14,산채류,99,기타,16,삿갓나물,1499,149916
2069,14,산채류,99,기타,17,병풍취,1499,149917


In [14]:
# DB로 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='master_crop_variety',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 산지코드-직팜코드-관측소코드 매핑

In [15]:
df = pd.read_csv('datasets/산지코드_직팜_관측지점_매핑완료_수정.csv', encoding='cp949')

In [16]:
def expand_row(row):
    if '~' in str(row['산지코드']):
        start, end = map(int, row['산지코드'].split('~'))
        return [
            {**row, '산지코드': code}
            for code in range(start, end+1)
        ]
    else:
        return [{**row, '산지코드': int(row['산지코드'])}]

# 예시 DataFrame: df
expanded = []
for _, row in df.iterrows():
    expanded.extend(expand_row(row))

df_expanded = pd.DataFrame(expanded)
df = df_expanded.rename(columns={'산지코드' : 'plor_cd',
                            '산지이름': 'plor_nm',
                            '직팜산지코드': 'j_sanji_cd', 
                            '직팜산지이름': 'j_sanji_nm', 
                            '관측지점' : 'station_cd'
                           })
df

,plor_cd,plor_nm,j_sanji_cd,j_sanji_nm,위도,경도,station_cd
0,100000,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
1,100001,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
2,100002,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
3,100003,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
4,100004,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
...,...,...,...,...,...,...,...
800030,971000,경상남도 창원시,1120,경상남도 창원시,35.2372,128.6811,255.0
800031,980000,경상북도,1169,경상북도,36.5681,128.7293,136.0
800032,981000,경상북도 포항시,1159,경상북도 포항시,36.0194,129.3434,138.0
800033,990000,제주도,1170,제주특별자치도,33.4996,126.5312,184.0


In [19]:
df.drop(columns=['위도', '경도'], inplace=True)

In [ ]:
df['station_cd'] = df['station_cd'].astype(int)

In [20]:
df.to_csv('datasets/map_region_weather_station_utf-8.csv', encoding='utf-8', index=False)

In [21]:
# 데이터베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='map_region_weather_station',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 일별 기상 데이터 

In [79]:
df = pd.read_csv('datasets/기상청_서울_일기요소_20180101-20250531.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257907 entries, 0 to 257906
Data columns (total 56 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TM             257907 non-null  int64  
 1   STN            257907 non-null  int64  
 2   WS_AVG         257907 non-null  float64
 3   WR_DAY         257907 non-null  int64  
 4   WD_MAX         257907 non-null  int64  
 5   WS_MAX         257907 non-null  float64
 6   WS_MAX_TM      257907 non-null  int64  
 7   WD_INS         257907 non-null  int64  
 8   WS_INS         257907 non-null  float64
 9   WS_INS_TM      257907 non-null  int64  
 10  TA_AVG         257907 non-null  float64
 11  TA_MAX         257907 non-null  float64
 12  TA_MAX_TM      257907 non-null  int64  
 13  TA_MIN         257907 non-null  float64
 14  TA_MIN_TM      257907 non-null  int64  
 15  TD_AVG         257907 non-null  float64
 16  TS_AVG         257907 non-null  float64
 17  TG_MIN         257907 non-nul

In [80]:
df= df.loc[:, ['TM', 'STN', 'TA_AVG', 'TA_MAX', 'TA_MIN', 'HM_AVG', 'RN_DAY', 'RN_60M_MAX']]
# 강수량, 1시간최고 강수량은 결측치(-9) 혹은 비가 안옴(0)이 많아 0 이하는 0으로 처리
# merged_df['강수량(mm)'] = merged_df['강수량(mm)'<=0].count()
df.loc[df['RN_DAY']<=0, 'RN_DAY'] = 0
df.loc[df['RN_60M_MAX']<=0, 'RN_60M_MAX'] = 0
df['TM'] = pd.to_datetime(df['TM'].astype(str), format='%Y%m%d')
df.to_csv('datasets/weather_daily.csv', encoding='utf-8', index=False)

In [29]:
#데이터베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='weather_daily',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 외생변수 테이블

In [30]:
import pandas as pd
# 외생요소 가공
df_factor_external = pd.read_csv('datasets/holiday_작기.csv', encoding='utf-8')

# 멜팅
df_melted = df_factor_external.melt(
    id_vars=['date', 'holiday_flag', 'holiday_score'],  # 고정 컬럼
    var_name='crop_name',         # 새로 생길 품목명 컬럼명
    value_name='grow_score'       # 각 품목의 작기지수 값 컬럼명
)

# 3. 필요하면 중복 제거
df_melted = df_melted.drop_duplicates()

# 4. 필요하면 인덱스 리셋
df = df_melted.reset_index(drop=True)

         date  holiday_flag  holiday_score crop_name  grow_score
0  2018-01-01             1            0.0        배추         1.0
1  2018-01-02             0            0.0        배추         1.0
2  2018-01-03             0            0.0        배추         1.0
3  2018-01-04             0            0.0        배추         1.0
4  2018-01-05             0            0.0        배추         1.0


In [31]:
# 품목
# 1. 품목명 → 품목코드 매핑 딕셔너리 수동 작성
crop_code_map = {
    "양파": "1201",  
    "배추": "1001", 
    "상추": "1005",  
    "과수": "0601",   
    "무" : "1101",
#    '배' : '0602',
    "마늘" : "1209",
    "건고추" : "1207",
    "감자" : "0501",
}

# 2. crop_name → crop_code로 변환 컬럼 추가
df['item_code'] = df['crop_name'].map(crop_code_map)
df.drop(colums='crop_name', inplace=True)

# 3. 중간 저장 (데일리) - 혹시나 쓰거나 기준이 바뀔수 있으니 백업
df.to_csv('datasets/factor_external_daily.csv', encoding='utf-8', index=False)

In [32]:
# 주간 병합하기
# 주차 식별 칼럼 만들기
def get_week_of_year(date):
    date = pd.to_datetime(date)
    year = date.year
    week_number = date.isocalendar().week
    return f"{year}{week_number:02d}"

df['week_no'] = df['date'].apply(get_week_of_year)

# 중복 제거
df.drop_duplicates(inplace=True)
df.drop(columns='date', inplace=True)

# 주차와 코드로 병합하기
df = df.groupby(['week_no', 'item_code']).sum(['holiday_flag', 'holiday_score', 'grow_score']).reset_index().sort_values(by='week_no', ascending=True)
df.to_csv('datasets/factor_external_weekly.csv', encoding='utf-8', index=False)

In [33]:
# 데이터 베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='factor_external_weekly',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 거래데이터 삽입

In [71]:
# 양파부터 시작 - 원본에서 출발
df1 = pd.read_csv('datasets/유통공사_도매시장_양파_20180103-20250531.csv', encoding='cp949')
df2 = pd.read_csv('datasets/유통공사_retry_양파_1.csv', encoding='cp949')
df3 = pd.read_csv('datasets/유통공사_retry_양파_2(완).csv', encoding='cp949')
df = pd.concat([df1, df2, df3], axis=0)
df.drop_duplicates(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15140\1300820689.py:2: DtypeWarning: Columns (9,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('datasets/유통공사_도매시장_양파_20180103-20250531.csv', encoding='cp949')
C:\Users\user\AppData\Local\Temp\ipykernel_15140\1300820689.py:4: DtypeWarning: Columns (9,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('datasets/유통공사_retry_양파_2(완).csv', encoding='cp949')


In [72]:
# 직팜코드 테이블 소환 - 향후 이걸 DB로 가져오자
df_region = pd.read_csv('datasets/map_region_weather_station_utf-8.csv', encoding='utf-8')
# 직팜코드 붙이기
df_region['plor_cd'] = df_region['plor_cd'].astype(str)
df_merged_1 = pd.merge(df, df_region[['plor_cd', 'j_sanji_cd']],  on='plor_cd', how='left')
df_merged_1['trd_clcln_ymd'] = pd.to_datetime(df_merged_1['trd_clcln_ymd'], format='%Y-%m-%d')

In [73]:
# plor_cd가 문자열이 아닐 가능성 대비
df['plor_cd'] = df['plor_cd'].fillna('').astype(str)

pattern = r'^[^0-9]+$'

condition = (
    df['totprc'].isna() | (df['totprc'] <= 0) |
    df['unit_tot_qty'].isna() | (df['unit_tot_qty'] <= 0) |
    df['plor_cd'].str.strip().isin(['0', '0.0']) |
    df['plor_cd'].str.match(pattern, na=False) |
    df['plor_nm'].isna() |
    (df['plor_nm'] == 0)
)

# 조건에 해당하는 행 추출
df_filtered = df[~condition]

# 수입산을 하나로 몰까 했지만.. 그냥 패스
# df.loc[df['plor_cd'].str.startswith('800')]['plor_cd'] = '800000'

# 아이템 풀코드 장착
for col in ['gds_lclsf_cd', 'gds_mclsf_cd', 'gds_sclsf_cd']:
    df_merged_1[col] = df_merged_1[col].astype(str) 
    df_merged_1[col] = df_merged_1[col].str.zfill(2)
df_merged_1['crop_full_code'] = df_merged_1['gds_lclsf_cd']+df_merged_1['gds_mclsf_cd']+df_merged_1['gds_sclsf_cd']

# 필요한 열만 선별

df = df_merged_1[['trd_clcln_ymd', 'crop_full_code', 'j_sanji_cd', 'unit_tot_qty', 'totprc']]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1027777 entries, 0 to 1027776
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   trd_clcln_ymd   1027777 non-null  datetime64[ns]
 1   crop_full_code  1027777 non-null  object        
 2   j_sanji_cd      975700 non-null   float64       
 3   unit_tot_qty    1027777 non-null  float64       
 4   totprc          1027777 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 39.2+ MB


,trd_clcln_ymd,crop_full_code,j_sanji_cd,unit_tot_qty,totprc
0,2018-01-03,120199,1133.0,4200.0,4893000.0
1,2018-01-03,120199,1000.0,15000.0,13500000.0
2,2018-01-03,120199,1133.0,4200.0,3976000.0
3,2018-01-03,120199,1133.0,2100.0,1610000.0
4,2018-01-03,120199,1133.0,2400.0,1860000.0


In [74]:
# 데이터 머지 (일, 작물코드, 산지)

df_merged = df.groupby(['trd_clcln_ymd', 'crop_full_code', 'j_sanji_cd']).sum(['unit_tot_qty', 'totprc']).reset_index() 
# df_merged = df_merged.sort_values('trd_clcln_ymd')
df_merged

,trd_clcln_ymd,crop_full_code,j_sanji_cd,unit_tot_qty,totprc
0,2018-01-03,120100,1022.0,7740.0,6558500.0
1,2018-01-03,120100,1079.0,375.0,546000.0
2,2018-01-03,120100,1085.0,7200.0,6430000.0
3,2018-01-03,120100,1086.0,6000.0,6610000.0
4,2018-01-03,120100,1093.0,1060.0,1228900.0
...,...,...,...,...,...
245800,2025-05-31,120199,1121.0,2720.0,1470000.0
245801,2025-05-31,120199,1122.0,4580.0,2507000.0
245802,2025-05-31,120199,1123.0,120.0,72000.0
245803,2025-05-31,120199,1144.0,738.0,419800.0


In [78]:
df_merged.to_csv('datasets/fact_trade.csv', encoding='utf-8', index=False)

In [77]:
# 데이터 베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df_merged.to_sql(
    name='fact_trade',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!
